In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from diffusers import StableDiffusionPipeline
import torch

In [2]:
# 1. Load the LLaMA or similar open LLM (here we use TinyLLaMA for speed)
llm_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(llm_name)
model = AutoModelForCausalLM.from_pretrained(llm_name, torch_dtype=torch.float16, device_map="auto")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
# 2. Generate an image prompt using the LLM
prompt_instruction = "Generate a vivid image prompt for a fantasy landscape:"
inputs = tokenizer(prompt_instruction, return_tensors="pt").to(model.device)
output_ids = model.generate(**inputs, max_new_tokens=50, do_sample=True, temperature=0.8)
generated_prompt = tokenizer.decode(output_ids[0], skip_special_tokens=True).split(":")[-1].strip()

print(f"🎯 Prompt for image generation: {generated_prompt}")

🎯 Prompt for image generation: A vast, foreboding forest stretched out as far as the eye could see, with towering trees that seemed to stretch their branches to the sky. Beyond the trees, the landscape was a maze of towering mountains,


In [4]:
# 3. Use Stable Diffusion to generate image from the prompt
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
).to("cuda")


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
image = pipe(generated_prompt).images[0]

# 4. Save the image
image.save("generated_image.png")
print("🖼️ Image saved as generated_image.png")

  0%|          | 0/50 [00:00<?, ?it/s]

🖼️ Image saved as generated_image.png
